## Exploratory Data Review

In [0]:
%sql
SELECT * FROM uci_raw.sensus LIMIT 5

In [0]:
%sql
SELECT a.sex, COUNT(*) 
FROM uci_raw.sensus a
GROUP BY a.sex

Databricks visualization. Run in Databricks to view.

Here we can see the first problem already.
We have in our dataset values such as '<=50K' and '<=50.''
This period at the end is enough to create a new category and make a mess in our model

In [0]:
%sql
SELECT b.sex, b.income, COUNT(*) as total FROM uci_raw.sensus b
GROUP BY b.sex, b.income 

In [0]:

%sql
SELECT DISTINCT income
FROM uci_raw.sensus
WHERE income RLIKE '\\.$|^\\s+|\\s+$';

Adjusted visualization

In [0]:
%sql
-- remove ponto final, tira espaços e padroniza em maiúsculas
SELECT
  b.sex,
  TRIM(UPPER(REGEXP_REPLACE(b.income, '\\.$', ''))) AS income_norm,
  COUNT(*) AS total
FROM uci_raw.sensus b
GROUP BY b.sex, TRIM(UPPER(REGEXP_REPLACE(b.income, '\\.$', '')))
ORDER BY b.sex, total DESC;

Databricks visualization. Run in Databricks to view.

Adjusted Data:

In [0]:
%sql
UPDATE uci_raw.sensus
SET income = TRIM(REGEXP_REPLACE(income, '\\.$', ''))
WHERE income RLIKE '\\.$|^\\s+|\\s+$';

After Adjusting the data we can see that now the visuzlization is the same

In [0]:
%sql
SELECT b.sex, b.income, COUNT(*) as total FROM uci_raw.sensus b
GROUP BY b.sex, b.income

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT c.sex, c.education, COUNT(*) AS total 
FROM uci_raw.sensus c
GROUP BY c.sex, c.education
ORDER BY c.sex, total DESC


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
WITH clean AS (
  SELECT
    sex,
    TRIM(UPPER(REGEXP_REPLACE(income, '\\.$', ''))) AS income_norm
  FROM uci_raw.sensus
)
SELECT
  sex,
  100.0 * SUM(CASE WHEN income_norm = '>50K' THEN 1 ELSE 0 END) / COUNT(*) AS pct_gt50k
FROM clean
GROUP BY sex
ORDER BY sex;

Databricks visualization. Run in Databricks to view.